# Tester

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.utils.data as data
import torchvision
from tqdm import tqdm
import torch.nn.functional as F
import math

from vit_pooling import ViTPooling


gpu_ids = []
device_names = []
if torch.cuda.is_available():
    for gpu_id in range(torch.cuda.device_count()):
        gpu_ids += [gpu_id]
        device_names += [torch.cuda.get_device_name(gpu_id)]
print(gpu_ids)
print(device_names)

if len(gpu_ids) > 1:
    gpu = 'cuda:' + str(gpu_ids[1])  # GPU Number
else:
    gpu = "cuda" if torch.cuda.is_available() else "cpu"

[0, 1, 2, 3]
['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']


In [2]:
device = gpu
model_path = './save/ViT_i2012_ep300_lr0.00005.pt'
BATCH_SIZE = 512
NUM_EPOCHS = 300
NUM_WORKERS = 2
LEARNING_RATE = 0.003

IMAGE_SIZE = 224
PATCH_SIZE = 16
IN_CHANNELS = 3
NUM_CLASSES = 1000
EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 12

In [3]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# pre_train_set = torchvision.datasets.ImageFolder('./data/ImageNet-21k', transform=transform_train)
# pre_train_loader = data.DataLoader(pre_train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
train_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/train', transform=transform_train)
train_loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/val', transform=transform_test)
test_loader = data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [4]:
# class TesterTimm(object):
#     def __init__(self):
#         self.model = None
#         self.optimizer = None
#         self.scheduler = None
#         self.epochs = []
#         self.losses = []

#     def process(self):
#         self.build_model()
#         self.eval_model()

#     def build_model(self):
#         self.model = timm.models.vit_base_patch16_224(pretrained=True).to(device)
#         self.model.num_classes = NUM_CLASSES
#         self.optimizer = Adam(self.model.parameters(), lr=LEARNING_RATE)
        
#         checkpoint = torch.load(model_path)
#         self.epochs = checkpoint['epochs']
#         self.model.load_state_dict(checkpoint['model'])
#         self.optimizer.load_state_dict(checkpoint['optimizer'])
#         self.losses = checkpoint['losses']
#         print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
#         print(f'Classes: {self.model.num_classes}')
#         print(f'Epochs: {self.epochs[-1]}')
        
#     def eval_model(self):
#         self.model.eval()

#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for i, data in tqdm(enumerate(testloader, 0), total=len(testloader)):
#                 images, labels = data
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = self.model(images)
#                 _, predicted = torch.max(outputs.data, 1)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
#         print(f'Accuracy {len(testset)} test images: {100 * correct / total:.2f} %')

In [5]:
class TesterPaper(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.epochs = []
        self.losses = []
        self.cls_token = None

    def process(self):
        self.build_model()
        self.eval_model()
        
    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Sampling Loss: {i:.3f}') for i in self.losses]
        

    def build_model(self):
        self.model = ViTPooling(image_size=IMAGE_SIZE,
                         patch_size=PATCH_SIZE,
                         in_channels=IN_CHANNELS,
                         num_classes=NUM_CLASSES,
                         embed_dim=EMBED_DIM,
                         depth=DEPTH,
                         num_heads=NUM_HEADS,
                         ).to(device)
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {NUM_CLASSES}')
        print(f'Epochs: {self.epochs[-1]}')

    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy {len(test_set)} test images: {100 * correct / total:.2f} %')

In [6]:
if __name__ == '__main__':
    t = TesterPaper()
    t.loss_checker()

Parameter: 100493032
Classes: 1000
Epochs: 33
Steps: 655k steps
Sampling Loss: 6.527
Sampling Loss: 6.074
Sampling Loss: 5.916
Sampling Loss: 5.663
Sampling Loss: 5.650
Sampling Loss: 5.309
Sampling Loss: 5.344
Sampling Loss: 5.265
Sampling Loss: 5.195
Sampling Loss: 4.979
Sampling Loss: 4.964
Sampling Loss: 5.048
Sampling Loss: 4.825
Sampling Loss: 4.504
Sampling Loss: 4.398
Sampling Loss: 4.318
Sampling Loss: 4.834
Sampling Loss: 4.541
Sampling Loss: 4.588
Sampling Loss: 4.481
Sampling Loss: 4.275
Sampling Loss: 3.889
Sampling Loss: 4.155
Sampling Loss: 4.315
Sampling Loss: 4.792
Sampling Loss: 4.061
Sampling Loss: 4.162
Sampling Loss: 4.169
Sampling Loss: 3.936
Sampling Loss: 4.668
Sampling Loss: 3.522
Sampling Loss: 3.895
Sampling Loss: 4.120
Sampling Loss: 3.819
Sampling Loss: 4.323
Sampling Loss: 3.983
Sampling Loss: 4.109
Sampling Loss: 3.788
Sampling Loss: 3.838
Sampling Loss: 4.405
Sampling Loss: 3.678
Sampling Loss: 3.583
Sampling Loss: 3.704
Sampling Loss: 3.819
Sampling Los

In [ ]:
if __name__ == '__main__':
    t = TesterPaper()
    t.process()

Parameter: 100493032
Classes: 1000
Epochs: 33


  1%|          | 1/98 [00:06<10:01,  6.20s/it]

In [ ]:
# 1스텝 = 9분
# 1에포크 = 3시간